Now I start to perform some feature engineering.

From the standford car dataset web site, I can find the bounding box information from the devkit zip file.
Using this bounding box, I can find out exactly where is the car from the image.
By cropping out the bounding box from the image, I can remove all the unneccesarily noise from the image and allow the resnet model to focus only on the car.


In [3]:
import cv2 as cv
import numpy as np
import pandas as pd
import scipy.io
import os
import shutil
import random
import tarfile


In [4]:
def untar(fname):
    if (fname.endswith("tgz")):
        tar = tarfile.open(fname)
        tar.extractall()
        tar.close()
        print ("Extracted in Current Directory")
    else:
        print ("Not a tgz file")
        
train_path = "cars_train.tgz"

untar(train_path)

test_path = "cars_test.tgz"

untar(test_path)

Extracted in Current Directory
Extracted in Current Directory


In [5]:
meta_path = "car_devkit.tgz"

untar(meta_path)

Extracted in Current Directory


In [6]:
def ensure_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

'''
Save the training dataset after getting the bounding box image from original training dataset.
'''
def save_train_data(fnames, bboxes):
    src_folder ='cars_train'
    num_samples = len(fnames)



    for i in range(num_samples):
        fname = fnames[i]
        (x1, y1, x2, y2) = bboxes[i]

        src_path = os.path.join(src_folder, fname)
        src_image = cv.imread(src_path)
        height, width = src_image.shape[:2]
        # margins of 16 pixels
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)

        dst_path = 'color_train'

        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        dst_path = os.path.join(dst_path, fname)

        crop_image = src_image[y1:y2, x1:x2]
        dst_img = cv.resize(src=crop_image, dsize=(img_height, img_width))
        cv.imwrite(dst_path, dst_img)

'''
Save the testing dataset after getting the bounding box image from original testing dataset.
'''
def save_test_data(fnames, bboxes):
    src_folder = 'cars_test'
    
    num_samples = len(fnames)


    for i in range(num_samples):
        dst_path = 'color_test'
        fname = fnames[i]
        (x1, y1, x2, y2) = bboxes[i]
        src_path = os.path.join(src_folder, fname)
        src_image = cv.imread(src_path)
        height, width = src_image.shape[:2]
        # margins of 16 pixels
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)
        
        dst_path = os.path.join(dst_path, fname)

        crop_image = src_image[y1:y2, x1:x2]
        dst_img = cv.resize(src=crop_image, dsize=(img_height, img_width))
        cv.imwrite(dst_path, dst_img)

'''
Getting the bounding box information for training dataset from devkit zip file.
After that, call save_train_data to extract the bounding box.
'''
def process_train_data():
    print("Processing train data...")
    cars_annos = scipy.io.loadmat('devkit\cars_train_annos')
    annotations = cars_annos['annotations']
    annotations = np.transpose(annotations)

    fnames = []
    class_ids = []
    bboxes = []
    labels = []

    for annotation in annotations:
        bbox_x1 = annotation[0][0][0][0]
        bbox_y1 = annotation[0][1][0][0]
        bbox_x2 = annotation[0][2][0][0]
        bbox_y2 = annotation[0][3][0][0]
        
        fname = annotation[0][5][0]
        bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
        fnames.append(fname)


    save_train_data(fnames, bboxes)
    print("Done process train data.")
    
'''
Getting the bounding box information for testing dataset from devkit zip file.
After that, call save_test_data to extract the bounding box.
'''
def process_test_data():
    print("Processing test data...")
    cars_annos = scipy.io.loadmat('devkit\cars_test_annos')
    annotations = cars_annos['annotations']
    annotations = np.transpose(annotations)

    fnames = []
    bboxes = []

    for annotation in annotations:
        bbox_x1 = annotation[0][0][0][0]
        bbox_y1 = annotation[0][1][0][0]
        bbox_x2 = annotation[0][2][0][0]
        bbox_y2 = annotation[0][3][0][0]
        fname = annotation[0][4][0]
        bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
        fnames.append(fname)

    save_test_data(fnames, bboxes)
    print("Done process test data.")



'''
This is the output image width and height after getting the bounding box. 
'''
img_width, img_height = 224, 224

ensure_folder("color_train")
ensure_folder("color_test")


process_train_data()
process_test_data()


Processing train data...
Done process train data.
Processing test data...
Done process test data.
